Script to analyse ChEMBL drug resistant variants

In [21]:
import pandas as pd
fin="/Users/anight/Documents/external presentations/EBI-Outreach/2018/UniProt-PDBe-workshop/Cases-studies/Human_variants_ChEMBL.tsv"
indf = pd.read_csv(fin, sep="\t")
indf.values

array([[638, 'D400A', 'Q9Y5X5', 1.0],
       [481, 'Y321A', 'Q9Y5X5', 1.0],
       [1102, 'E310A', 'Q9Y5X5', 1.0],
       ..., 
       [305, 'H86F', 'O14842', nan],
       [167, 'L186F', 'O14842', nan],
       [667, 'S422D', 'O00141', 1.0]], dtype=object)

In [60]:
import pandas as pd
import re
import time
import requests, sys
import ijson
import json
import math


def checkSequence(seq: str, position: int, wildtype: str) -> bool:
        if seq.find(wildtype,position):
            return True
        return False

def checkResponse(resp) -> bool:
    if(resp.status_code == 200):
        return True
    else:
        return False  
 
def checkNoData(resp) -> bool:
    if(resp.status_code == 404):
        return True
    else:
        return False

    
def queryAPI(url: str,acc: str) -> []:
        urlFormat = "format=json"
        
        fullUrl = url + "/" + acc + "?" + urlFormat 
        #print(fullUrl)
        responseFail = 0
        response = requests.get(fullUrl) 
        if(checkNoData(response)):
            response.close()
            return []
        
        while(not checkResponse(response)):
            if(responseFail == 10):
                break
            # Record a failed response from server
            else:
                responseFail += 1
                time.sleep(2)
                response = requests.get(fullUrl)
        
        jsonResponse = response.json()
        
        response.close()
        return jsonResponse    
    
    

    
def parseFeautesAndXrefs(quacc: str, mutdata: [], pos: int) -> []: 
    #A/D=2-1068 or A/C=227-461, B/D=133-188
    rxp = re.compile('\d+')
    url = "https://www.ebi.ac.uk/proteins/api/proteins"
    protresp = queryAPI(url,quacc)
    if "features" in protresp:
        feats = []
        for ft in protresp["features"]:
            #if quacc == 'Q9NTG7':
            #    print(ft)
            if not ft["begin"] == 'unknown' and not ft["end"] == 'unknown' \
            and pos >= int(ft["begin"]) and pos <= int(ft["end"]):
                feats.append(ft)
        mutdata.append(feats)
    else:
        mutdata.append([])
    if "dbReferences" in protresp:
        xrefs = []
        for xr in protresp["dbReferences"]:
            if "PDB" == xr["type"]:
                if "chains" in xr["properties"]: 
                    chains = xr["properties"]["chains"]
                    for ch in chains.split(','):
                        crange = rxp.findall(ch)
                        if pos >= int(crange[0]) and pos <= int(crange[1]):
                            xrefs.append(xr["id"])
        mutdata.append(xrefs)
    else:
        mutdata.append([])
    return mutdata
    
    
    
def checkForVariant(varfts: [],wt: str,pos: str,vt: str) -> []:
    for ft in varfts:
        if ft["type"] == "VARIANT" and ft["begin"] == pos \
        and ft["wildType"] == wt and ft["alternativeSequence"] == vt:
            return ft
# wt = R, alt = * and begin = 4 

def updateforbadseqmatch(mutdata: []) -> []:
    mutdata.append('fail')
    mutdata.append('fail')
    mutdata.append([])
    mutdata.append(False)
    mutdata.append([])
    

# Main here 
resCVS = "/Users/anight/Documents/external presentations/EBI-Outreach/2018/UniProt-PDBe-workshop/Cases-studies/ChEMBL_drug_resist_anal_results.csv"
rxppos = re.compile('\d+')
rxp = re.compile('([A-Z])(\d+)([A-Z])')

rescols = ['accession', 'wildtype', 'muttype', 'position', 'seqpass', 'knownvariant', 'varinatIDs','disease',
           'annotation','features','pdbe']

resdf = pd.DataFrame(columns=rescols)
varcnt = 0
url = "https://www.ebi.ac.uk/proteins/api/variation"

fin="/Users/anight/Documents/external presentations/EBI-Outreach/2018/UniProt-PDBe-workshop/Cases-studies/Human_variants_ChEMBL.tsv"
indf = pd.read_csv(fin, sep="\t")

#for r in indf.iterrows():
for r in indf.values:
    print(r)
    quacc = r[2]

    # requires isoform?
    if not math.isnan(r[3]) and int(r[3]) != 1:
        quacc = quacc + '-' + str(int(r[3]))

    # 'S631A,L666A'
    for mut in r[1].split(','):
        mutdata = []
        mutdata.append(quacc)
        rxpm = rxp.match(mut)
        wt = rxpm.group(1)
        pos = rxpm.group(2)
        vt = rxpm.group(3)
        mutdata.append(wt)
        mutdata.append(vt)
        mutdata.append(pos)

        varresp = queryAPI(url,quacc)
        if(len(varresp) == 0):
            matchedSeq = False
        else:
            vrseq = varresp["sequence"]
            matchedSeq = checkSequence(vrseq,int(pos),wt)

        if matchedSeq:
            mutdata.append('pass')
            var = checkForVariant(varresp["features"],wt,pos,vt)
            if var is not None:
                mutdata.append('pass')
                vxrf = []
                for xr in var["xrefs"]:
                    vxrf.append(xr["id"])
                vxrfset = set(vxrf)
                vxrf = list(vxrfset)
                mutdata.append(vxrf)
                assd = []
                if "association" in var:
                    mutdata.append(True)
                    for ass in var["association"]:
                        assd.append(ass["name"])
                        assevid = []
                        for evid in ass["evidences"]:
                            assevid.append(evid["source"])
                        assd.append(assevid)
                        if "xrefs" in ass:
                            assd.append(ass["xrefs"])
                    mutdata.append(assd)
                else:
                    mutdata.append(False)
                    mutdata.append([])
            else:
                mutdata.append('fail')
                mutdata.append([])
                mutdata.append(False)
                mutdata.append([])

            # Next step is to get features and PDB xrefs
            mutdata = parseFeautesAndXrefs(quacc, mutdata, int(pos))

        else:
            mutdata = updateforbadseqmatch(mutdata)
        resdf.loc[varcnt] = mutdata
        varcnt+=1

    
resdf.to_csv(resCVS, index=False)
print(resdf)

[638 'D400A' 'Q9Y5X5' 1.0]
[481 'Y321A' 'Q9Y5X5' 1.0]
[1102 'E310A' 'Q9Y5X5' 1.0]
[1239 'Q410A' 'Q9Y5X5' 1.0]
[639 'Y417A' 'Q9Y5X5' 1.0]
[482 'Y415A' 'Q9Y5X5' 1.0]
[656 'D355N' 'Q9Y468' 4.0]
[827 'D355A' 'Q9Y468' 4.0]
[459 'T36E' 'Q9Y2R2' 1.0]
[615 'C129S' 'Q9Y2R2' 1.0]
[171 'S35E,T36E' 'Q9Y2R2' 1.0]
[170 'L29A' 'Q9Y2R2' 1.0]
[775 'R33A' 'Q9Y2R2' 1.0]
[934 'K138A' 'Q9Y2R2' 1.0]
[933 'F28A' 'Q9Y2R2' 1.0]
[307 'S35E' 'Q9Y2R2' 1.0]
[1273 'S472D' 'Q9Y243' 1.0]
[111 'F89G' 'Q9UQM7' nan]
[110 'T286D' 'Q9UQM7' nan]
[1240 'H1006Y' 'Q9UQL6' nan]
[747 'C1156Y' 'Q9UM73' nan]
[748 'G1269A' 'Q9UM73' nan]
[1063 'G1202R' 'Q9UM73' nan]
[283 'R1275Q' 'Q9UM73' nan]
[1210 'L1152R' 'Q9UM73' nan]
[437 'F1174L' 'Q9UM73' nan]
[749 'L1196M' 'Q9UM73' nan]
[1171 'C645A' 'Q9UM07' nan]
[101 'R374A' 'Q9UM07' nan]
[259 'Q346E' 'Q9UM07' nan]
[260 'R374K' 'Q9UM07' nan]
[875 'Q346A' 'Q9UM07' nan]
[410 'C553S' 'Q9UL62' nan]
[714 'C558S' 'Q9UL62' nan]
[1055 'I1061T' 'Q9UHC9' 1.0]
[426 'S808G' 'Q9P2K8' 1.0]
[735 'L195F' 

[1209 'I259W' 'P61073' 1.0]
[1217 'A175F' 'P61073' 1.0]
[137 'H281A' 'P61073' 1.0]
[144 'V280A' 'P61073' 1.0]
[145 'G207W' 'P61073' 1.0]
[281 'N176A' 'P61073' 1.0]
[282 'E275A' 'P61073' 1.0]
[295 'S263A' 'P61073' 1.0]
[296 'D171N' 'P61073' 1.0]
[431 'V112A' 'P61073' 1.0]
[432 'L120F' 'P61073' 1.0]
[433 'D181A' 'P61073' 1.0]
[434 'Y255A' 'P61073' 1.0]
[435 'D262A' 'P61073' 1.0]
[444 'W283A' 'P61073' 1.0]
[583 'D171A' 'P61073' 1.0]
[584 'F172A' 'P61073' 1.0]
[585 'H203A' 'P61073' 1.0]
[586 'G207F' 'P61073' 1.0]
[600 'T287A' 'P61073' 1.0]
[601 'Q200W' 'P61073' 1.0]
[603 'I284A' 'P61073' 1.0]
[741 'V99A' 'P61073' 1.0]
[742 'E277A' 'P61073' 1.0]
[914 'D262N' 'P61073' 1.0]
[217 'C175A,R40C' 'P58335' 1.0]
[833 'R40C' 'P58335' 1.0]
[1160 'H976Y' 'P56524' nan]
[1026 'E200A' 'P54098' nan]
[1185 'Y126T' 'P53609' 1.0]
[1186 'F52Y,Y126T' 'P53609' 1.0]
[271 'F52Y,F53Y,Y126T' 'P53609' 1.0]
[836 'T210D' 'P53350' nan]
[1040 'A133D' 'P52732' nan]
[881 'D130V,A133D' 'P52732' nan]
[562 'I299F' 'P52732' na

[1256 'K1250A' 'P13569' 1.0]
[498 'G551D' 'P13569' 1.0]
[919 'S345C' 'P12931' 1.0]
[750 'T341M' 'P12931' 1.0]
[1198 'R274L' 'P11473' 1.0]
[1197 'H397A' 'P11473' 1.0]
[428 'S235A' 'P11473' 1.0]
[740 'K240A' 'P11473' 1.0]
[739 'Q239A' 'P11473' 1.0]
[579 'H305A' 'P11473' 1.0]
[578 'D149A' 'P11473' 1.0]
[522 'V561M' 'P11362' 1.0]
[803 'Y381A' 'P11229' 1.0]
[693 'K57A' 'P11086' nan]
[358 'V559D,V654A' 'P10721' 1.0]
[357 'V559D' 'P10721' 1.0]
[824 'K642E' 'P10721' 1.0]
[825 'T670I' 'P10721' 1.0]
[504 'D816V' 'P10721' 1.0]
[505 'V559D,T670I' 'P10721' 1.0]
[506 'V560G' 'P10721' 1.0]
[503 'D816H' 'P10721' 1.0]
[989 'L576P' 'P10721' 1.0]
[1262 'A829P' 'P10721' 1.0]
[507 'V654A' 'P10721' 1.0]
[515 'W31A' 'P10599' 1.0]
[1068 'L552P' 'P10253' nan]
[752 'S529V,S619R' 'P10253' nan]
[596 'G549R' 'P10253' nan]
[141 'T373K' 'P08913' nan]
[278 'S119A' 'P08684' nan]
[1093 'Y1230C' 'P08581' 1.0]
[313 'M1250T' 'P08581' 1.0]
[463 'N1100Y' 'P08581' 1.0]
[179 'Y1235D' 'P08581' 1.0]
[314 'Y1235H' 'P08581' 1.0]


In [84]:
import requests
r = requests.get('https://www.ebi.ac.uk/proteins/api/variation/P61073?from=0&size=10&format=json') 
c = r.headers.get('content-type')
print(c)
a = r.headers.get("X-Pagination-TotalRecords")
print(a)

application/json
None
